# Movie genre prediction with Object2Vec Algorithm

1. [Introduction](#Introduction)
2. [Install and import dependencies](#Install-and-import-dependencies)
3. [Preprocessing](#Preprocessing)
  1. [Build the vocabulary](#Build-the-vocabulary)
  2. [Split data into train, validation and test](#Split-data-into-train,-validation-and-test)
  3. [Negative sampling](#Negative-sampling)
  4. [Tokenization](#Tokenization)
  5. [Download pretrained word embeddings](#Download-pretrained-word-embeddings)
4. [Sagemaker Training](#Sagemaker-Training)
  1. [Upload data to S3](#Upload-data-to-S3)
  1. [Training hyperparameters](#Training-hyperparameters)
5. [Evaluation with Batch inference](#Evaluation-with-Batch-inference)
6. [Online inference demo](#Online-inference-demo)

## Introduction

In this notebook, we will explore how ObjectToVec algorithm can be used in a multi label prediction setting 
to predict the genre of a movie from its plot description. We will be using a dataset provided from imdb.


At a high level, the network architecture that we use for this task is illustrated in the diagram below.

<img src="image.png" width="500">

We cast the problem of multi-label prediction as a binary classification problem. A positive example is the tuple of movie plot description, and a movie genre that applies to the movie in the labeled data. If a movie has multiple genres, we create multiple positive examples for the movie, one for each genre. A negative example is a pair where the genre does not apply to the movie. The negative examples are generated by picking a random subset of genres which do not apply to the movie, as determined by the labeled dataset.

Let us first start with downloading the data.

<div class="alert alert-warning">
Important: Before you begin downloading, please read the following README file using your browser and make sure you are okay with the license.
ftp://ftp.fu-berlin.de/pub/misc/movies/database/frozendata/README
</div>

In [ ]:
!wget ftp://ftp.fu-berlin.de/pub/misc/movies/database/frozendata/genres.list.gz
!gunzip genres.list.gz

In [ ]:
!wget ftp://ftp.fu-berlin.de/pub/misc/movies/database/frozendata/plot.list.gz
!gunzip plot.list.gz

## Install and import dependencies

In [ ]:
!pip install langdetect
!pip install nltk
!conda upgrade -y sqlite
!pip install jsonlines

In [ ]:
import json
import sys
from collections import Counter
from collections import defaultdict
from itertools import chain, islice

import boto3
import jsonlines
import matplotlib
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import sagemaker
import seaborn as sns
from langdetect import detect
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer, sent_tokenize
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input
from sklearn.model_selection import StratifiedShuffleSplit

%matplotlib inline

## Preprocessing

In [ ]:
def get_genres(filename):
    genres = defaultdict(list)
    unique_genres = set()
    with open(filename, "r", errors='ignore') as f:
        for line in f:
            if line.startswith('"'):
                data = line.split('\t')
                movie = data[0]
                genre = data[-1].strip()
                genres[movie].append(genre)
                unique_genres.add(genre)
    unique_genres = sorted(unique_genres)
    data = []
    for movie in genres:
        row = [0]*len(unique_genres)
        for g in genres[movie]:
            row[unique_genres.index(g)] = 1
        row.insert(0, movie)
        data.append(row)
    genres_df = pd.DataFrame(data)
    genres_df.columns = ['short_title'] + unique_genres
    return genres_df
    
genres_df = get_genres("genres.list")
genres_df.head()

In [ ]:
def get_plots(filename):
    with open(filename, "r", errors='ignore') as f:
        data = []
        inside = False
        plot = ''
        full_title = ''
        for line in f:
            if line.startswith("MV:") and not inside:
                inside = True
                full_title = line.split("MV:")[1].strip()

            elif line.startswith("PL:") and inside:
                plot += line.split("PL:")[1].replace("\n", "")

            elif line.startswith("MV:") and inside:
                short_title = full_title.split('{')[0].strip()
                data.append((short_title, full_title, plot))
                plot = ''
                inside = False
    plots_df = pd.DataFrame(data)
    plots_df.columns = ['short_title', 'title', 'plot']
    return plots_df

plots_df = get_plots("plot.list")
plots_df.head()

Now join the genre and the plot dataframes.

In [ ]:
data_df = plots_df.merge(genres_df, how='inner', on='short_title')
data_df.dropna(inplace=True)
data_df.drop('short_title', axis=1, inplace=True)
data_df.head()

In [ ]:
genres = list(data_df.columns)[2:]
counts = []
for genre in genres:
    counts.append((genre, data_df[genre].sum()))
distribution = pd.DataFrame(counts, columns=['genre', 'count'])
distribution

In [ ]:
# Remove the genres with 0 movies
data_df.drop('Lifestyle', axis=1, inplace=True)
data_df.drop('Sci-fi', axis=1, inplace=True)

Next we select all the movies whose description are in English. Note that this will take about 12 minutes to run.

In [ ]:
data_df['plot_lang'] = data_df.apply(lambda row: detect(row['plot']), axis=1)
data_df['plot_lang'].value_counts()

In [ ]:
df = data_df[data_df.plot_lang.isin(['en'])]
df.to_csv("movies_genres_en.csv", sep='\t', encoding='utf-8')

In [ ]:
df = pd.read_csv("movies_genres_en.csv", delimiter='\t', encoding='utf-8', index_col=0)

### Build the vocabulary

Lets define a few functions to tokenize our data and build the vocabulary.

In [ ]:
nltk.download('punkt')
tokenizer = TreebankWordTokenizer()

def tokenize_plot_summary(summary):
    for sent in sent_tokenize(summary):
        for token in tokenizer.tokenize(sent):
            yield token

UNKNOWN = '<unk>'
def build_vocab(data, max_vocab_size=None):
    vocab = Counter()
    total = len(data)
    for i, row in enumerate(data.itertuples()):
        vocab.update(tokenize_plot_summary(row.plot))
        if (i+1)%1000 == 0:
            sys.stdout.write(".")
            sys.stdout.flush()
    final_vocab = {word:i for i, (word, count) in enumerate(vocab.most_common(max_vocab_size))}
    final_vocab[UNKNOWN]=len(final_vocab)+1
    return final_vocab

In [ ]:
vocab = build_vocab(df)
print("Vocab size: ", len(vocab))
with open("vocab.json", "w") as f:
    json.dump(vocab, f)
    print("Saved vocabulary file to vocab.json")

### Split data into train, validation and test

Now we show how to prepare the data for training. First we define a function to convert a dataframe into a jsonlines format which can be used by the algorithm to train.

First we split the dataframe into train, validation and test partitions.

In [ ]:
def split(df, test_size):
    data = df.values
    data_y = df.drop(['title', 'plot', 'plot_lang'], axis=1).values
    #StratifiedShuffleSplit does not work with one hot encoded / multiple labels. Doing the split on basis of arg max labels.
    data_y = np.argmax(data_y, axis=1)
    data_y.shape
    stratified_split = StratifiedShuffleSplit(n_splits=2, test_size=test_size, random_state=42)
    for train_index, test_index in stratified_split.split(data, data_y):
        train, test = df.iloc[train_index], df.iloc[test_index]
    return train, test

train, test = split(df, 0.33)
#Split the train further into train and validation
train, validation = split(train, 0.2)

### Negative sampling

The object2vec algorithm is setup as a binary classification problem. The true examples are the movie, genre pairs present in the dataset. In order to train the algorithm, we also need to provide negative examples. One option is to add all the genres to which the movie does not belong. However this strategy will create a highly skewed dataset with large percentage of negative example, as there are 27 classes present. Instead we choose to have 5 negative examples per positive example, as has been reported in related works like word2vec.

Lets look at the class distribution and figure out the how much we should sample the negative examples to achieve a balanced distribution of positive and negative examples.

In [ ]:
genres = list(train.columns)[2:-1]
print ("Number of genres: ", len(genres))
agg = {genre:'sum' for genre in genres}
agg_by_genre = train.agg(agg)
agg_by_genre

In [ ]:
total_positive_samples = agg_by_genre.sum()
total_negative_samples = len(train)*len(genres) - total_positive_samples

NEGATIVE_TO_POSITIVE_RATIO = 5
sampling_percent = NEGATIVE_TO_POSITIVE_RATIO * total_positive_samples / total_negative_samples
print("total positive examples: ", total_positive_samples)
print("total negative samples", total_negative_samples)
print("negative sampling needed: ", sampling_percent )

### Tokenization

Now we can proceed to create the tokenized jsonlines dataset for training, validation and test partitions. We will use negative sampling of 0.4 for the training set, and add all the negatives for validation and test sets.

In [ ]:
nltk.download('stopwords')
def tokenize(df, vocab, filename, negative_frac=1.0, use_stopwords=False):
    # Rename the columns so that they are valid python identifiers
    df = df.rename(lambda x:x.replace("-", "_") ,axis='columns')
    genres = list(df.columns)[2:-1]
    max_seq_length = 0
    total = len(df)
    stop_words = set()
    if use_stopwords:
        stop_words = set(stopwords.words('english'))
    with jsonlines.open(filename, mode='w') as writer:
        for j, row in enumerate(df.itertuples()):
            tokens = [token for token in tokenize_plot_summary(row.plot) if token not in stop_words]
            plot_token_ids = [vocab[token] if token in vocab else vocab[UNKNOWN] for token in tokens]
            for i, genre in enumerate(genres):
                label = getattr(row, genre)
                if label == 1 or np.random.rand() < negative_frac:
                    # All positive examples and fraction of negative examples are picked.
                    writer.write({"in0": plot_token_ids, "in1": [i], "label": label})
            max_seq_length = max(len(plot_token_ids), max_seq_length)
            if (j+1)%1000==0:
                sys.stdout.write(".")
                sys.stdout.flush()
        print("Finished tokenizing data. Max sequence length of the tokenized data: ", max_seq_length)

In [ ]:
tokenize(df=train, vocab=vocab, filename="tokenized_movie_genres_train.jsonl", negative_frac=0.4, use_stopwords=True)

In [ ]:
tokenize(df=validation, vocab=vocab, filename="tokenized_movie_genres_validation.jsonl", use_stopwords=True)

In [ ]:
tokenize(df=test, vocab=vocab, filename="tokenized_movie_genres_test.jsonl", use_stopwords=True)

For better performance, the training dataset needs to be shuffled.

In [ ]:
!shuf tokenized_movie_genres_train.jsonl > tokenized_movie_genres_train_shuffled.jsonl

### Download pretrained word embeddings

We will make use of pretrained word embeddings from https://nlp.stanford.edu/projects/glove/. 

<div class="alert alert-warning">
Important: Before you begin downloading, please read the following  and make sure you are okay with the license.
https://opendatacommons.org/licenses/pddl/1.0/
</div>

In [ ]:
!mkdir /tmp/glove
!wget -P /tmp/glove/ http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip -d /tmp/glove /tmp/glove/glove.840B.300d.zip
!rm /tmp/glove/glove.840B.300d.zip

## Sagemaker Training

Let us start with defining some configurations 

In [ ]:
bucket='<bucket-name>' # customize to your bucket

In [ ]:
prefix = 'object2vec-movie-genre-prediction'

container = get_image_uri(boto3.Session().region_name, 'object2vec')

train_s3_path = "s3://{}/{}/data/train/".format(bucket, prefix)
validation_s3_path = "s3://{}/{}/data/validation/".format(bucket, prefix)
test_s3_path = "s3://{}/{}/data/test/".format(bucket, prefix)
auxiliary_s3_path = "s3://{}/{}/data/auxiliary/".format(bucket, prefix)
prediction_s3_path = "s3://{}/{}/predictions/".format(bucket, prefix)

### Upload data to S3

In [ ]:
!aws s3 cp tokenized_movie_genres_train_shuffled.jsonl {train_s3_path}
!aws s3 cp tokenized_movie_genres_validation.jsonl {validation_s3_path}
!aws s3 cp tokenized_movie_genres_test.jsonl {test_s3_path}

In [ ]:
!aws s3 cp vocab.json {auxiliary_s3_path}
!aws s3 cp /tmp/glove/glove.840B.300d.txt {auxiliary_s3_path}

### Training hyperparameters

The object2vec is a customizable algorithm and hence it has quite a few hyperparameters. Lets review some of the important ones:

* **enc_dim**: The dimension of the encoder. Both the movie plot description and genre embeddings are mapped to this dimension. 
* **mlp_dim**: The dimension of the output from multilayer perceptron (MLP) layers.
* **mlp_activation**: Type of activation function for the multilayer perceptron (MLP) layer.
* **mlp_layers**: The number of multilayer perceptron (MLP) layers in the network.
* **output_layer**: The type of output layer. We choose 'softmax' as it is a classification problem.
* **bucket_width**: The allowed difference between data sequence length when bucketing is enabled. Bucketing is enabled when a non-zero value is specified for this parameter.
* **num_classes**: The number of classes for classification training, which is 2 for our case.

The **enc0** encodes the movie plot description which is a sequence, and **enc1** encodes the movie genre which is a single token. The encoder parameters:

* **max_seq_len**: The maximum sequence length that will be considered. Any input tokens beyond max_seq_len will be truncated and ignored. We choose a value of 500 for enc
* **network**: Network model. We choose hcnn for both enc0 and enc1.
* **cnn_filter_width**: The filter width of the hcnn encoder.
* **layers**: The number of layers. We choose 2 layers for enc0, as we want to capture richer structures in the movie plot description which is a sequence input. For enc1, we choose 1 layer.
* **token_embedding_dim**: The output dimension of  token embedding layer. We choose a dimension of 300 for encoder 0, consistent with the dimension of the glove embdeddings. For enc1, we choose 10.
* **pretrained_embedding_file**: The filename of pretrained token embedding file present in the auxiliary data channel. We use the glove embeddings for enc0. For enc1, the embeddings will be learned by the algorithm.
* **freeze_pretrained_embedding**: Whether to freeze  pretrained embedding weights. We set this to True for enc0.
* **vocab_file**: The vocabulary file for mapping pretrained token embeddings to vocabulary IDs. This is specified only for enc0, as we use pretrained embeddings only for enc0.
* **vocab_size**: The vocabulary size of the tokens. For enc0, it is the number of words appearing the dataset. For enc1, it is the number of genres.

In [ ]:
hyperparameters = {
 'enc_dim': 4096, 
 'mlp_dim': 512, 
 'mlp_activation': 'relu', 
 'mlp_layers': 2, 
 'output_layer': 'softmax',
 'bucket_width': 10, 
 'num_classes': 2,
 
 'mini_batch_size': 256,
 
 'enc0_max_seq_len': 500,
 'enc1_max_seq_len': 2,
 
 'enc0_network': 'hcnn',
 'enc1_network': 'hcnn',
    
 'enc0_layers': '2',
 'enc1_layers': '1',
    
 'enc0_cnn_filter_width': 2,
 'enc1_cnn_filter_width': 1,
 
 'enc0_token_embedding_dim': 300,
 'enc1_token_embedding_dim': 10,
 
 'enc0_pretrained_embedding_file' : "glove.840B.300d.txt",
 
 'enc0_freeze_pretrained_embedding': 'true',
 
 'enc0_vocab_file': 'vocab.json',
 'enc1_vocab_file': '',
 
 'enc0_vocab_size': len(vocab),
 'enc1_vocab_size': len(genres),
}


<div class="alert alert-warning">
Note that the training will take approximately 1.5 hours to complete on the ml.p2.8xlarge instance type
</div>


In [ ]:
o2v = sagemaker.estimator.Estimator(container,
                                    get_execution_role(), 
                                    train_instance_count=1, 
                                    train_instance_type='ml.p3.8xlarge',
                                    output_path="s3://{}/{}/output".format(bucket, prefix),
                                   )
o2v.set_hyperparameters(**hyperparameters)
input_data = {
    "train": s3_input(train_s3_path, content_type="application/jsonlines"),
    "validation": s3_input(validation_s3_path, content_type="application/jsonlines"),
    "auxiliary": s3_input(auxiliary_s3_path)
}
o2v.fit(input_data)

## Evaluation with Batch inference

<div class="alert alert-warning">
Note that the batch inference will take approximately 30 minutes to complete on the ml.p2.8xlarge instance type
</div>


In [ ]:
transformer = o2v.transformer(instance_count=1, 
                              instance_type="ml.p3.8xlarge", 
                              output_path=prediction_s3_path)
transformer.transform(data=test_s3_path, content_type="application/jsonlines", split_type="Line")
transformer.wait()

Download the predictions from s3 to perform the evaluation.

In [ ]:
!aws s3 cp --recursive {prediction_s3_path} .

In [ ]:
def evaluate(filename, predictions, genre_dict, threshold=0.5):
    metrics = {g:{"genre": g, "tp":0, "tn":0, "fp":0, "fn":0} for g in genre_dict.values()}
    with jsonlines.open(filename, "r") as reader, jsonlines.open(predictions, "r") as preds:
        for row, preds in zip(reader, preds):
            prediction = preds["scores"][1] > threshold
            label = row["label"]
            g = genre_dict[row["in1"][0]]
            if prediction == 1:
                if label == prediction:
                    metrics[g]["tp"] +=1
                else:
                    metrics[g]["fp"]+=1
            elif prediction == 0:
                if label == prediction:
                    metrics[g]["tn"]+=1
                else:
                    metrics[g]["fn"]+=1
    summary = pd.DataFrame(list(metrics.values())).set_index('genre')
    summary['accuracy'] = summary.apply (lambda row: (row.tp + row.tn) / (row.tp + row.tn + row.fp + row.fn),axis=1)
    summary['precision'] = summary.apply (lambda row: row.tp / (row.tp + row.fp),axis=1)
    summary['recall'] = summary.apply (lambda row: row.tp / (row.tp + row.fn),axis=1)
    summary['f1'] = summary.apply (lambda row: 2*(row.precision * row.recall) /(row.precision + row.recall),axis=1)
    return summary

In [ ]:
genre_dict = {i:genre for i, genre in enumerate(genres)}
summary =evaluate("tokenized_movie_genres_test.jsonl", "tokenized_movie_genres_test.jsonl.out", genre_dict, threshold=0.6)
summary

In [ ]:
tp_sum = summary["tp"].sum()
fp_sum = summary["fp"].sum()
tn_sum = summary["tn"].sum()
fn_sum = summary["fn"].sum()
precision = (tp_sum) / (tp_sum + fp_sum)
recall = (tp_sum) / (tp_sum + fn_sum)

print("Accuracy: ", (tp_sum + tn_sum) / (tp_sum + fp_sum + tn_sum + fn_sum))
print("Micro Precision: ", precision)
print("Micro Recall: ", recall)
print("Micro F1: ", 2*precision*recall/(precision + recall))

We compared the performance with [fastText](https://fasttext.cc/). Fasttext does not perform multi-label predictions, so to do a fair comparison we trained 28 binary classification models with fastText for each of the movie genres and combined the results of each predictor. While training the fastText models we set **wordNgrams** to 2, **dim** to 300 and  **pretrainedVectors** to the glove embeddings.

<img src="comparison.png">

## Online inference demo

In this section we setup a online inference endpoint and perform inference for a few recently released movies.

In [ ]:
predictor = o2v.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

In [ ]:
def get_movie_genre_predictions(movie_summary, genre_dict, vocab, predictor, threshold=0.5):
    plot_token_ids = [vocab[token] if token in vocab else vocab[UNKNOWN] for token in tokenize_plot_summary(movie_summary)]
    batch = [{"in0": plot_token_ids, "in1": [genre_id]} for genre_id in range(len(genre_dict))]
    request = {"instances": batch}
    response = predictor.predict(data=json.dumps(request))
    scores = [score["scores"] for score in json.loads(response)["predictions"]]
    predictions = [genre_dict[i] for i, score in enumerate(scores) if score[1] > threshold]
    return predictions

In [ ]:
star_trek = "Ten years before Kirk, Spock and the Enterprise, theUSS Discovery discovers new worlds and lifeforms \
as one Starfleet officer learns to understand all things alien."

get_movie_genre_predictions(star_trek, genre_dict, vocab, predictor)

In [ ]:
nun = "A priest with a haunted past and a novice on the threshold of her final vows are sent by the Vatican \
to investigate the death of a young nun in Romania and confront a malevolent force in the form of a demonic nun."
get_movie_genre_predictions(nun, genre_dict, vocab, predictor)

In [ ]:
fantastic_beasts = "The second installment of the 'Fantastic Beasts' series set in J.K. Rowling's Wizarding World \
featuring the adventures of magizoologist Newt Scamander."
get_movie_genre_predictions(fantastic_beasts, genre_dict, vocab, predictor)

In [ ]:
predictor.delete_endpoint()